[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.leafmap.org/lab/index.html?path=notebooks/85_gedi.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/leafmap/blob/master/docs/notebooks/85_gedi.ipynb)
[![image](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/opengeos/leafmap/HEAD)

**How to search and download GEDI L4A dataset interactively**

This notebook shows how to search and download the [GEDI L4A dataset](https://daac.ornl.gov/cgi-bin/dsviewer.pl?ds_id=2056) using [leafmap](https://leafmap.org). The source code is adapted from the [gedi_tutorials](https://github.com/ornldaac/gedi_tutorials) repository but has been greatly simplified. Credit goes to ORNL DAAC and [Rupesh Shrestha](https://github.com/rupesh2).


Uncomment the following line to install [leafmap](https://leafmap.org) if needed.

In [ ]:
# %pip install -U "leafmap[vector]"

In [ ]:
from leafmap import leafmap

Create an interactive map.

In [ ]:
m = leafmap.Map(height=600)
m

Use the draw tool to draw a rectangle on the map. If no rectangle is drawn, the default bounding box will be used.

In [ ]:
if m.user_roi is not None:
    roi = m.user_roi_bounds()
else:
    roi = [-73.9872, -33.7683, -34.7299, 5.2444]

Specify the start and end date.

In [ ]:
start_date = "2020-07-01"
end_date = "2020-07-31"

Search and download GEDI L4A dataset.

In [ ]:
gdf = leafmap.gedi_search(roi, start_date, end_date, add_roi=False, sort_filesize=True)
gdf.head()

Visualize the GEDI L4A dataset footprints.

In [ ]:
m.add_gdf(gdf, layer_name="GEDI footprints")
m

Download the GEDI L4A dataset from NASA EarthData website. You need to register an account first if you don't have one. Create an account at https://urs.earthdata.nasa.gov. Then return to this notebook and uncomment the following code cell to set your username and password.

In [ ]:
# import os
# os.environ["EARTHDATA_USERNAME"] = "your_username"
# os.environ["EARTHDATA_PASSWORD"] = "your_password"

In [ ]:
leafmap.gedi_download_files(gdf.head(), outdir="data")

Read the downloaded GEDI L4A dataset as a GeoDataFrame.

In [ ]:
try:
    gdf = leafmap.h5_to_gdf(
        "data/*.h5", dataset="BEAM0110", columns=["agbd"], nodata=-9999
    )
except:
    # Download the sample data if the above code fails
    gdf = leafmap.geojson_to_gdf(
        "https://github.com/opengeos/data/releases/download/v1.0.0/gedi_sample.geojson"
    )
gdf.head()

We can subset the GeoDataFrame by a bounding box. First, create an interactive map and add the bounding box to the map.

In [ ]:
m = leafmap.Map(height=620)
roi = [-38.8641, -6.8664, -37.2107, -6.359]
bbox = leafmap.bbox_to_gdf(roi)
m.add_gdf(bbox, layer_name="ROI", zoom_to_layer=True, info_mode=None)
m

Use the bounding box to subset the GeoDataFrame.

In [ ]:
subset = leafmap.filter_bounds(gdf, roi, within=True)

Add the subsetted GeoDataFrame to the map. Note that this is only for visualizing a small subset of the data. If you want to work with the entire dataset, you can skip this step.

In [ ]:
m.add_data(subset, column="agbd", cmap="Greens", marker_radius=5)

Visualize the GEDI L4A Aboveground Biomass Density (AGBD) data with lonboard.

In [ ]:
import leafmap

In [ ]:
leafmap.view_vector(
    gdf, color_column="agbd", color_map="Greens", color_k=10, get_radius=25
)